#### Loading and preparing the PCam data for training deep learning models using tensorflow dataset (tfds)

In [1]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"   # Delete if you have GPU's available
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

import tensorflow_datasets as tfds

2023-01-24 20:34:51.507417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/gustavchristensen/Documents/SDU/MSc. Data Science/3. Semester - DT/Anvendt Maskinlæring/applied_ML_faelles/assignment_2/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Defining a function that splits images and labels and one-hot-encodes the labels

In [5]:
def convert_sample(sample):
    image, label = sample['image'], sample['label']  
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, label

Run the code to change the data directory to your local folder

In [3]:
# GUSTAV
data_dir = '/Users/gustavchristensen/Documents/SDU/MSc. Data Science/3. Semester - DT/Anvendt Maskinlæring'

In [ ]:
# ANDREAS
data_dir =

In [ ]:
# DAVID
data_dir =

In [ ]:
# CHRISTOFFER K
data_dir =

In [ ]:
# CHRISTOFFER S
data_dir =

Next we use the tensorflow dataset API - tfds - to load data from your mounted google drive. Note this API requite that you should have copied the entire **patch_camelyon** folder from https://syddanskuni-my.sharepoint.com/:f:/g/personal/cmd_sam_sdu_dk/EiWD2LmuxCJBp-_tfGK7aL8Bair7l5z8FU5sp5pLjlhKwg?e=FLzWno to the /content/drive/MyDrive folder on your google drive:

In [6]:
ds1,ds2,ds3 = tfds.load('patch_camelyon',split=['train[:10%]','test[:10%]','validation[:10%]'],
                        data_dir = data_dir,
                        download=False,
                        shuffle_files=True)

Next we simple transform the data (by the function convert sample described previously) and getting ready for training by splitting it into batches.

In [7]:
train_dataset       = ds1.map(convert_sample).batch(32)
validation_dataset  = ds3.map(convert_sample).batch(32)
test_dataset        = ds2.map(convert_sample).batch(32)

The data is then ready to be applied for training, validation, testing etc...below just a very very simple illustration on how to construct and train a model based on the data we have prepared

In [8]:
def first_ccn_model():
    input_img = Input(shape=(96,96,3))
    
    x = Conv2D(16, (3, 3), padding='valid', activation='relu')(input_img)
    x = Conv2D(32, (3, 3), padding='valid', activation='relu')(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    y = Dense(2, activation='softmax')(x)

    model = Model(inputs=input_img, outputs=y)
    return model

sgd_opt = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model = first_ccn_model()

model.compile(optimizer=sgd_opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

hist = model.fit(train_dataset,
                 validation_data=validation_dataset,
                 epochs=2)

Epoch 1/2
820/820 [==============================] - 513s 623ms/step - loss: 0.6912 - accuracy: 0.5279 - val_loss: 0.6960 - val_accuracy: 0.4931
Epoch 2/2
820/820 [==============================] - 404s 493ms/step - loss: 0.6935 - accuracy: 0.5043 - val_loss: 0.6956 - val_accuracy: 0.4931
